### TEST cases of computing a equilibrium

Here I'm following the definition in the notes. So the goal is to find prices ($w_i, R_i$) such that:

- the hh problem is solved
- the distribution is consistent with the hh problem and stationary
- markets clear so there is the goods market condition equation (26) and bond market condition (27)

Note here that the wage is sufficent in that all $p_{ij}$ are pinned down from this.

A couple of details in my code that confused me

- I set everything up so country ones wage is the numerarie and only 2*Ncntry-1 markets are looked to be cleared. 

- the ``aggregate`` function constructs production (valued at price $p_i$) and then imports bilaterally. 

- then in ``world_equillibrium`` around line 70, I construct the matrix of trade flows with a row being the destination country, and then working across columns one has the countries that that country is buying from. 

- what previous bullet implies is that when equation (26) is computed, a countries exports (including to itself) are the sum down a row. Where as the sum across columns are imports. Line 25 in code discusses this as well.

---


In [2]:
include("../code/julia/ha-trade-environment.jl")

include("../code/julia/ha-trade-solution.jl") #in here is the ``world_equillibrium'' function

include("../code/julia/ha-trade-helper-functions.jl") #in there is the ``aggregate'' function

include("../code/julia/static-trade-environment.jl")
using MINPACK

---

### Symmetric 4 countries

In [3]:
Ncntry = 4

dtest = 2.0
d = dtest.*ones(Ncntry,Ncntry)
d[diagind(d)] .= 1.0

TFP = [1.0, 1.0, 1.00, 1.00]

mdl_prm = world_model_params(Ncntry = Ncntry, Na = 100, Nshocks = 5, 
γ = 2.0, ϕ = 3, amax = 8.0, σ = 0.40, ρ = 0.20, σϵ = 0.25, d = d, TFP = TFP)

# not sure why these values

@unpack Na, Nshocks, Ncntry, TFP = mdl_prm

R = 1.03*ones(Ncntry);
W = 1.0*ones(Ncntry);

In [4]:
f(x) = world_equillibrium(x, mdl_prm, hh_solution_method = "itteration");

function f!(fvec, x)

    fvec .= f(x)

end

initial_x = [W[2:end]; R]

n = length(initial_x)
diag_adjust = n - 1

sol = fsolve(f!, initial_x, show_trace = true, method = :hybr;
      ml=diag_adjust, mu=diag_adjust,
      diag=ones(n),
      mode= 1,
      tol=1e-5,
       )

print(sol)

Wsol = [1.0; sol.x[1:Ncntry-1]]
Rsol = sol.x[Ncntry:end]

Y, tradeflows, A_demand, tradeshare, hh, dist = world_equillibrium(Rsol, Wsol, mdl_prm, hh_solution_method = "itteration");

Iter     f(x) inf-norm    Step 2-norm      Step time
------   --------------   --------------   --------------
     1     2.780362e-01     0.000000e+00        40.965000
     2     3.629802e-02     1.949642e-05         9.829000
     3     2.327439e-03     3.378723e-06         1.079000
     4     1.231814e-04     7.502279e-09         1.043000
     5     2.377496e-05     1.098913e-12         1.132000
Results of Nonlinear Solver Algorithm
 * Algorithm: Modified Powell
 * Starting Point: [1.0, 1.0, 1.0, 1.03, 1.03, 1.03, 1.03]
 * Zero: [0.9999999997835487, 0.9999999997833852, 0.9999999997834562, 1.0317823752547763, 1.0317822035599291, 1.0317822035602404, 1.0317822035601023]
 * Inf-norm of residuals: 0.000024
 * Convergence: true
 * Message: algorithm estimates that the relative error between x and the solution is at most tol
 * Total time: 54.049000 seconds
 * Function Calls: 5
 * Jacobian Calls (df/dx): 1

In [5]:
Y

4-element Vector{Float64}:
 1.1043882543572114
 1.1043882541181695
 1.1043882541179877
 1.1043882541180692

In [6]:
tradeflows

4×4 Matrix{Float64}:
 0.932536   0.0572844  0.0572844  0.0572844
 0.0572842  0.932535   0.0572842  0.0572842
 0.0572842  0.0572842  0.932535   0.0572842
 0.0572842  0.0572842  0.0572842  0.932535

So this look pretty good. Converged quickly, the wages are all equal or $\approx$ 1.0 and then the interest rates are the same. 

---

### 4 Country asymmetric 

In [7]:
Ncntry = 4

dtest = 2.0
d = dtest.*ones(Ncntry,Ncntry)
d[diagind(d)] .= 1.0

TFP = [1.0, 1.0, 0.75, 1.25] 
#here there is a rich and a poor country

mdl_prm = world_model_params(Ncntry = Ncntry, Na = 100, Nshocks = 5, 
γ = 2.0, ϕ = 3, amax = 8.0, σ = 0.40, ρ = 0.20, σϵ = 0.25, d = d, TFP = TFP)

@unpack Na, Nshocks, Ncntry, TFP = mdl_prm

R = 1.03*ones(Ncntry);
W = 1.0*ones(Ncntry);


In [8]:
f(x) = world_equillibrium(x, mdl_prm, hh_solution_method = "itteration");

function f!(fvec, x)

    fvec .= f(x)

end

initial_x = [W[2:end]; R]

n = length(initial_x)
diag_adjust = n - 1

sol = fsolve(f!, initial_x, show_trace = true, method = :hybr;
      ml=diag_adjust, mu=diag_adjust,
      diag=ones(n),
      mode= 1,
      tol=1e-5,
       )

print(sol)

Wsol = [1.0; sol.x[1:Ncntry-1]]
Rsol = sol.x[Ncntry:end]

Y, tradeflows, A_demand, tradeshare, hh, dist = world_equillibrium(Rsol, Wsol, mdl_prm, hh_solution_method = "itteration");

Iter     f(x) inf-norm    Step 2-norm      Step time
------   --------------   --------------   --------------
     1     4.519892e-01     0.000000e+00         0.881000
     2     2.074719e-01     3.424722e-02         7.353000
     3     1.484559e-01     1.320159e-03         1.040000
     4     1.608679e-01     1.479236e-04         0.983000
     5     8.422125e-02     9.746771e-06         0.967000
     6     4.090719e-02     2.244559e-06         0.973000
     7     1.143755e-03     2.449194e-07         0.957000
     8     1.756105e-04     1.393589e-10         1.036000
Results of Nonlinear Solver Algorithm
 * Algorithm: Modified Powell
 * Starting Point: [1.0, 1.0, 1.0, 1.03, 1.03, 1.03, 1.03]
 * Zero: [0.9999999103444615, 0.8280999397832072, 1.1533952485591796, 1.0317702664236748, 1.0317702574042107, 1.0339885565644116, 1.0295060762744703]
 * Inf-norm of residuals: 0.000176
 * Convergence: true
 * Message: algorithm estimates that the relative error between x and the solution is at mos

In [9]:
Y

4-element Vector{Float64}:
 1.1043882543572199
 1.1043881553426895
 0.9145438469304947
 1.273796165140177

In [10]:
tradeflows

4×4 Matrix{Float64}:
 0.928934   0.0570974  0.0351285  0.0832275
 0.0570974  0.928934   0.0351285  0.0832275
 0.0348608  0.0348608  0.788773   0.0560552
 0.0834949  0.0834949  0.0555191  1.05129

So far this passes the smell test. The rich country is richer, people buy more stuff from it. The poor country is poor and people buy less stuff from it. Also notice that country 1 and 2 are the same, so the wage in country 2 is $\approx$ 1 and then their interest rates are virtually the same. 